看了前面几个视频，给出的notebook都不是很好，都不能运行，终于这次视频给出的代码是可运行的了，本文主要是介绍怎么使用tf来对文章做summary，很cool的功能，也很实用，想想我自己读完一篇文章后都不能做总结，现在机器可以做了，真是令人兴奋

本文使用的数据集是：https://metamind.io/research/the-wikitext-long-term-dependency-language-modeling-dataset/

这些数据都是从Wikipedia的文章上解析出来的100million的tokens

In [ ]:
# dependencies

import numpy as np #vectorization
import random #generate probability distribution 
import tensorflow as tf #ml
import datetime #clock training time

下一步我们将读取文章内容,数据的下载地址是：https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip

In [ ]:
text = open('wiki.test.raw').read()
print('text length in number of characters:', len(text))

print('head of text:')
print(text[:100]) #all tokenized words, stored in a list called text

下一步我们要将数据进行排序，看有多少的字符

In [ ]:
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of characters:', char_size)
print(chars)

下一步我们要建立两个字典，一个是index => character，另一个是character => index

In [ ]:
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
def sample(prediction):
    #Samples are uniformly distributed over the half-open interval 
    r = random.uniform(0,1)
    #store prediction char
    s = 0
    #since length > indices starting at 0
    char_id = len(prediction) - 1
    #for each char prediction probabilty
    for i in range(len(prediction)):
        #assign it to S
        s += prediction[i]
        #check if probability greater than our randomly generated one
        if s >= r:
            #if it is, thats the likely next char
            char_id = i
            break
    #dont try to rank, just differentiate
    #initialize the vector
    char_one_hot = np.zeros(shape=[char_size])
    #that characters ID encoded
    #https://image.slidesharecdn.com/latin-150313140222-conversion-gate01/95/representation-learning-of-vectors-of-words-and-phrases-5-638.jpg?cb=1426255492
    char_one_hot[char_id] = 1.0
    return char_one_hot

接着我们要做的是怎么将输入转换为向量

在lstm中都会有个连续输入的值，然后导出一个输出，形象点就是：
![](http://karpathy.github.io/assets/rnn/diags.jpeg)

在[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)文章中有介绍具体的模型，我们有多个输入来预测一个输出

In [ ]:
len_per_section = 50
skip = 2
sections = []
next_chars = []
#fill sections list with chunks of text, every 2 characters create a new 50 
#character long section
#because we are generating it at a character level
for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])

In [ ]:
print(sections[0],"\nPredictions:",next_chars[0])

In [ ]:
print(sections[1],"\nPredictions:",next_chars[1])

我们可以看到sections[1]比sections[0]就是多移动了skip=2位，然后每len_per_section=50预测1个character

下一步我们就是将输入和输出转换为one-hot encoded的形式：

In [ ]:
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1

batch size的意义：理论上我们应该将所有的数据都放入模型中计算后，然后通过梯度方法进行更新，但是这样带来的问题是：内存消耗太大了，于是我们就有了部分数据后就进行梯度更新的方法，这个batch size就是一次更新中用到的数据

> batch size = the number of training examples in one forward/backward pass

该值越大，消耗的内存也就越多



In [ ]:
batch_size = 512
#total iterations
max_steps = 72001
#how often to log?
log_every = 100
#how often to save?
save_every = 6000

#too few and underfitting
#Underfitting occurs when there are too few neurons 
#in the hidden layers to adequately detect the signals in a complicated data set.
#too many and overfitting
hidden_nodes = 1024
#starting text
test_start = 'I am thinking that'
#to save our model
checkpoint_directory = 'ckpt'



In [ ]:
#Create a checkpoint directory
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per epoch:', int(len(X)/batch_size))

下面我们会用tf的基本函数来构建lstm模型，关于lstm模型的理解我之前写过一篇文章：[lstm的理解](http://www.jianshu.com/p/2f33b3040b50)

最重要的概念是我们要实现3个门：
1. 遗忘门（forget gate）
2. 输入门（input gate）
3. 输出门（output gate）
![](http://upload-images.jianshu.io/upload_images/2256672-bff9353b92b9c488.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 遗忘门（forget gate）
遗忘门控制着以前的记忆有多少进行保存
![](https://cdn-images-1.medium.com/max/1600/1*9ZgZxRSVAvJEsaoWtz69yg.png)

针对这个图我们就有定义：

```
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
  
    forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
```

## 输入门
输入门控制着当前输入有多少进行记忆
![](https://cdn-images-1.medium.com/max/1600/1*lNDzNHVxLSKJEpCsP4KD4w.png)

## 输出门
输出门控制着当前状态有多少输出到输出
![](https://cdn-images-1.medium.com/max/1600/1*mOhp-z4KM0Qm6Y0RY7YgMQ.png)


下面我们来看完整的代码：

!note：关于初始化参数的选择问题可以看：[weight_initialization](https://github.com/zhuanxuhit/deep-learning/blob/4427bf70640baa16b9d0f4ffa2acdcff8dc7b50a/weight-initialization/weight_initialization.ipynb)

In [ ]:
graph = tf.Graph()
with graph.as_default():
    # 记录着当前步数
    global_step = tf.Variable(0)
    #data tensor shape feeding in sections
    data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
    #labels
    labels = tf.placeholder(tf.float32, [batch_size, char_size])
    
    # 记录各个参数，用 truncated_normal 来记录
    # input gate
    w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_io = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_i = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Forget gate: weights for input, weights for previous output, and bias
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Output gate: weights for input, weights for previous output, and bias
    w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_oo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_o = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Memory cell: weights for input, weights for previous output, and bias
    w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_co = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_c = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    def lstm(i, o, state):
        
        #these are all calculated seperately, no overlap until....
        #(input * input weights) + (output * weights for previous output) + bias
        input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
        #(input * forget weights) + (output * weights for previous output) + bias
        forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
        #(input * output weights) + (output * weights for previous output) + bias
        output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
        #(input * internal state weights) + (output * weights for previous output) + bias
        memory_cell = tf.tanh(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
        
        #...now! multiply forget gate * given state    +  input gate * hidden state
        state = forget_gate * state + input_gate * memory_cell
        #squash that state with tanh nonlin (Computes hyperbolic tangent of x element-wise)
        #multiply by output
        output = output_gate * tf.tanh(state)
        #return 
        return output, state
    
    # 具体的lstm的执行
    #both start off as empty, LSTM will calculate this
    output = tf.zeros([batch_size, hidden_nodes])
    state = tf.zeros([batch_size, hidden_nodes])
    
    #for each input set
    for i in range(len_per_section):
        #calculate state and output from LSTM
        output, state = lstm(data[:, i, :], output, state)
        #to start, 
        if i == 0:
            #store initial output and labels
            outputs_all_i = output
            labels_all_i = data[:, i+1, :]
        #for each new set, concat outputs and labels
        elif i != len_per_section - 1:
            #concatenates (combines) vectors along a dimension axis, not multiply
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, data[:, i+1, :]], 0)
        else:
            #final store
            outputs_all_i = tf.concat(axis=0, [outputs_all_i, output])
            labels_all_i = tf.concat(axis=0, [labels_all_i, labels])
     

当我们计算完lstm后，我们要对输出在做个FC层，将其转换为我们需要的数据，即one-hot encoded的数据

In [ ]:
learning_rate = 10.0

In [ ]:
with graph.as_default():
    w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([char_size]))

    logits = tf.matmul(outputs_all_i, w) + b
    
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels_all_i))
    # sparse_softmax_cross_entropy_with_logits 是当labels是排外的，只有一个是有值的时候使用
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels_all_i))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    ###########
    #Test
    ###########
    test_data = tf.placeholder(tf.float32, shape=[1, char_size])
    test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
    test_state = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #Reset at the beginning of each test
    reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
                                test_state.assign(tf.zeros([1, hidden_nodes])))

    #LSTM
    test_output, test_state = lstm(test_data, test_output, test_state)
    test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)

## 训练
模型都定义好了，下面就是进行训练了，此处对于最后的一些训练数据，我们循环的用开头的数据进行补全

In [ ]:
#timew to train the model, initialize a session with a graph
with tf.Session(graph=graph) as sess:
    #standard init step
    tf.global_variables_initializer().run()
    offset = 0
    saver = tf.train.Saver()
    
    #for each training step
    for step in range(max_steps):
        
        #starts off as 0
        offset = offset % len(X)
        
        #calculate batch data and labels to feed model iteratively
        if offset <= (len(X) - batch_size):
            #first part
            batch_data = X[offset: offset + batch_size]
            batch_labels = y[offset: offset + batch_size]
            offset += batch_size
        #until when offset  = batch size, then we 
        else:
            #last part
            to_add = batch_size - (len(X) - offset)
            batch_data = np.concatenate((X[offset: len(X)], X[0: to_add]))
            batch_labels = np.concatenate((y[offset: len(X)], y[0: to_add]))
            offset = to_add
        
        #optimize!!
        _, training_loss = sess.run([optimizer, loss], feed_dict={data: batch_data, labels: batch_labels})
        
        if step % 10 == 0:
            print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))

            if step % save_every == 0:
                saver.save(sess, checkpoint_directory + '/model', global_step=step)

## 预测
最后我们来给定一段文字，看输出结果的


In [ ]:
test_start = 'I plan to make the world a better place '

with tf.Session(graph=graph) as sess:
    #init graph, load model
    tf.global_variables_initializer().run()
    model = tf.train.latest_checkpoint(checkpoint_directory)
    saver = tf.train.Saver()
    saver.restore(sess, model)

    #set input variable to generate chars from
    reset_test_state.run() 
    test_generated = test_start

    #for every char in the input sentennce
    for i in range(len(test_start) - 1):
        #initialize an empty char store
        test_X = np.zeros((1, char_size))
        #store it in id from
        test_X[0, char2id[test_start[i]]] = 1.
        #feed it to model, test_prediction is the output value
        _ = sess.run(test_prediction, feed_dict={test_data: test_X})

    
    #where we store encoded char predictions
    test_X = np.zeros((1, char_size))
    test_X[0, char2id[test_start[-1]]] = 1.

    #lets generate 500 characters
    for i in range(500):
        #get each prediction probability
        prediction = test_prediction.eval({test_data: test_X})[0]
        # one hot encode it
        # 此处我不理解为什么不直接用 prediction  argmax 来得到next_char呢？
        next_char_one_hot = sample(prediction)
        #get the indices of the max values (highest probability)  and convert to char
        next_char = id2char[np.argmax(next_char_one_hot)]
        #add each char to the output text iteratively
        test_generated += next_char
        #update the 
        test_X = next_char_one_hot.reshape((1, char_size))

    print(test_generated)